In [94]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; } div.text_cell_render { font-family: 'Human BBY Office'; font-size: 12pt; line-height: 145%;}</style>"))

import random
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

import sklearn as sk
from sklearn import model_selection

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [78]:
df = pd.read_csv('~/data/glass/glass.data', sep=",", header=None, encoding="ISO-8859-1")
df.columns = ['ID','RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','Type']
variables = ['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe']
y = df[['Type']]
X = df.drop(columns=['ID', 'Type'])
df.drop(columns=['ID'])

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.780,0.06,8.750,0.000,0.00,A
1,1.51761,13.89,3.60,1.36,72.730,0.48,7.830,0.000,0.00,A
2,1.51618,13.53,3.55,1.54,72.990,0.39,7.780,0.000,0.00,A
3,1.51766,13.21,3.69,1.29,72.610,0.57,8.220,0.000,0.00,A
4,1.51742,13.27,3.62,1.24,73.080,0.55,8.070,0.000,0.00,A
5,1.51596,12.79,3.61,1.62,72.970,0.64,8.070,0.000,0.26,A
6,1.51743,13.30,3.60,1.14,73.090,0.58,8.170,0.000,0.00,A
7,1.51756,13.15,3.61,1.05,73.240,0.57,8.240,0.000,0.00,A
8,1.51918,14.04,3.58,1.37,72.080,0.56,8.300,0.000,0.00,A
9,1.51755,13.00,3.60,1.36,72.990,0.57,8.400,0.000,0.11,A


In [182]:
def makeContextualBanditDataFrame(df):
    
    # calculate mean and sd for each column by type
    stat_df = df.groupby('Type').agg({'RI': [np.mean, np.std], 'Na':[np.mean, np.std], 'Mg':[np.mean, np.std],'Al':[np.mean, np.std],'Si':[np.mean, np.std],'K':[np.mean, np.std],
                                  'Ca':[np.mean, np.std],'Ba':[np.mean, np.std],'Fe':[np.mean, np.std]})

    mock_columns = variables + ['TypeShown','Reward']
    mock_df = pd.DataFrame(columns=mock_columns)
    types = df.Type.unique().tolist()
    
    # for each row:
    for index, row in df.iterrows():
        
        new_row = row[variables]
        new_row['TypeShown'] = row['Type']
        new_row['Reward'] = 1
        
        mock_df = mock_df.append(new_row, ignore_index = True)
        
        # for each type ID:
        for atype in types:
            if atype == row['Type']:
                continue
            
            if random.randint(0,1) == 0:
                continue
                
            # adjust each column by a random number bound by [mean-sd, mean+sd], reward = 0
            mock_row = row[variables]
            mock_row['TypeShown'] = atype
            mock_row['Reward'] = 0
            
            mock_row = mutate_mock_row(mock_row, row['Type'], stat_df)
            
            mock_df = mock_df.append(mock_row, ignore_index=True)
    
    # mutate data frame to add probability of showing a type to each row
    type_percentages = mock_df.groupby('TypeShown').agg({'TypeShown': 'count'}).apply(lambda x: x / float(x.sum()))
    percentages = type_percentages['TypeShown'].to_dict()
    mock_df['LikelihoodShown'] = mock_df.apply(lambda row: percentages.get(row.TypeShown), axis=1)
        
    return mock_df

In [183]:
def mutate_mock_row(row, target_type, stat_df):
    
    for stat in stat_df.columns.levels[0].tolist():
        mean = stat_df[stat]['mean'].get(target_type)
        std = stat_df[stat]['std'].get(target_type)
        stat_value = random.uniform(max(0, mean-std), mean+std)
        row[stat] = stat_value
        
    return row        

In [193]:
mock_df = makeContextualBanditDataFrame(df)
mock_df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,TypeShown,Reward,LikelihoodShown
0,1.521010,13.640000,4.490000,1.100000,71.780000,0.060000,8.750000,0.000000,0.000000,A,1,0.175538
1,1.519627,13.266606,3.736565,1.017047,72.791175,0.254933,9.037226,0.046212,0.083838,B,0,0.173273
2,1.520644,13.646304,3.675548,1.414755,72.086215,0.581988,9.262544,0.050124,0.110819,C,0,0.132503
3,1.519384,13.239187,3.792539,0.920458,72.718340,0.645211,8.864300,0.015945,0.020824,E,0,0.125708
4,1.519976,12.955011,3.397854,1.114936,72.905187,0.340384,8.831133,0.006214,0.074260,F,0,0.122310
5,1.519687,13.352943,3.633922,1.086179,73.109248,0.432994,9.091255,0.069880,0.085111,G,0,0.134768
6,1.517610,13.890000,3.600000,1.360000,72.730000,0.480000,7.830000,0.000000,0.000000,A,1,0.175538
7,1.517242,13.646920,3.576309,1.237760,72.336849,0.406186,8.518443,0.054967,0.032531,B,0,0.173273
8,1.517199,12.778656,3.438333,1.244805,72.426233,0.284363,9.217869,0.032868,0.038475,D,0,0.135900
9,1.518403,13.320463,3.593856,1.357250,72.496404,0.517483,8.796341,0.025592,0.006815,E,0,0.125708


In [200]:
mock_size = mock_df.shape[0]
mock_df['weights'] = mock_df.apply(lambda row: row.Reward * -1.0 / row.LikelihoodShown, axis=1)
mock_df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,TypeShown,Reward,LikelihoodShown,weights
0,1.521010,13.640000,4.490000,1.100000,71.780000,0.060000,8.750000,0.000000,0.000000,A,1,0.175538,-5.696774
1,1.519627,13.266606,3.736565,1.017047,72.791175,0.254933,9.037226,0.046212,0.083838,B,0,0.173273,-0.000000
2,1.520644,13.646304,3.675548,1.414755,72.086215,0.581988,9.262544,0.050124,0.110819,C,0,0.132503,-0.000000
3,1.519384,13.239187,3.792539,0.920458,72.718340,0.645211,8.864300,0.015945,0.020824,E,0,0.125708,-0.000000
4,1.519976,12.955011,3.397854,1.114936,72.905187,0.340384,8.831133,0.006214,0.074260,F,0,0.122310,-0.000000
5,1.519687,13.352943,3.633922,1.086179,73.109248,0.432994,9.091255,0.069880,0.085111,G,0,0.134768,-0.000000
6,1.517610,13.890000,3.600000,1.360000,72.730000,0.480000,7.830000,0.000000,0.000000,A,1,0.175538,-5.696774
7,1.517242,13.646920,3.576309,1.237760,72.336849,0.406186,8.518443,0.054967,0.032531,B,0,0.173273,-0.000000
8,1.517199,12.778656,3.438333,1.244805,72.426233,0.284363,9.217869,0.032868,0.038475,D,0,0.135900,-0.000000
9,1.518403,13.320463,3.593856,1.357250,72.496404,0.517483,8.796341,0.025592,0.006815,E,0,0.125708,-0.000000


In [201]:
import tensorflow as tf

dataset = tf.data.Dataset.from_tensor_slices((dict(mock_df), mock_df.columns))

TypeError: Expected binary or unicode string, got 1